In [3]:
# 환경
import os, random
import numpy as np
import torch

os.environ["TORCH_COMPILE_DISABLE"] = "1"  # 토치 컴파일 비활성

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device = torch.device("mps"); torch_dtype = torch.float16
else:
    device = torch.device("cpu"); torch_dtype = torch.float32

print(f"Device: {device}, dtype: {torch_dtype}")

Device: cuda, dtype: torch.bfloat16


In [4]:
# 모델 및 토크나이저 로드

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_ID = "openai/gpt-oss-20b"
REVISION = None

_tok = AutoTokenizer.from_pretrained(MODEL_ID, revision=REVISION, trust_remote_code=True, use_fast=True)
if _tok.pad_token is None:
    _tok.pad_token = _tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=REVISION,
    trust_remote_code=True,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=False,
    device_map=None,
)
model.to(device); model.eval()

_gen = pipeline("text-generation", model=model, tokenizer=_tok,
                device=0 if device.type=="cuda" else -1)

print("Model & tokenizer ready.")

c:\Users\Admin\anaconda3\envs\gptoss\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
You have loaded an FP4 model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model. To remove this warning, pass device_map = 'cuda'. 
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.35s/it]
Device set to use cuda:0


Model & tokenizer ready.


In [7]:
# -*- coding: utf-8 -*-
"""
NTL(Neurotransmitter Tendency List) 벡터라이저 — GPT-OSS 20B 기반
입력 문자열 -> [d, s, n, m]([도파민, 세로토닌, 노르에피네프린, 멜라토닌])

요구사항:
- LLM 학습(파인튜닝) 없이, 프롬프트만으로 동작
- 출력 형식 엄수: 소수(0.0~1.0), 길이 4
- GPT-OSS가 신경전달물질 개념을 이해할 수 있도록, 프롬프트에 정의 포함
- 파서/검증, 배치 지원 + 이유(reason) 라인 추가
"""

# =============================
# 환경
# =============================
import os, re, json, random
import numpy as np
import torch
from dataclasses import dataclass
from typing import List, Tuple, Dict, Any

os.environ["TORCH_COMPILE_DISABLE"] = "1"  # 토치 컴파일 비활성

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device = torch.device("mps"); torch_dtype = torch.float16
else:
    device = torch.device("cpu"); torch_dtype = torch.float32

print(f"Device: {device}, dtype: {torch_dtype}")

# =============================
# 모델 및 토크나이저 로드
# =============================
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_ID = "openai/gpt-oss-20b"
REVISION = None

_tok = AutoTokenizer.from_pretrained(MODEL_ID, revision=REVISION, trust_remote_code=True, use_fast=True)
if _tok.pad_token is None:
    _tok.pad_token = _tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=REVISION,
    trust_remote_code=True,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=False,
    device_map=None,
)
model.to(device); model.eval()

_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=_tok,
    device=0 if device.type == "cuda" else -1,
)

print("Model & tokenizer ready.")

# =============================
# 프롬프트 템플릿
# =============================
SYSTEM_PRIMER = (
    "너는 신경과학 보조자다. 아래 지시를 정확히 따른다.\n"
    "목표: 어떤 입력 문장을 신경전달물질 경향 벡터 NTL로 변환한다.\n"
    "NTL 포맷은 반드시 다음과 같다: "
    "[d, s, n, m]([도파민, 세로토닌, 노르에피네프린, 멜라토닌])\n"
    "여기서 d,s,n,m은 0.0~1.0 범위의 소수(최대 소수점 3자리)다.\n"
    "추가로 두 번째 줄에 반드시 '이유: '로 시작하는 간단한 설명을 한 문장으로 제공한다.\n\n"
    "정의 요약:\n"
    "- 도파민(dopamine): 동기/보상/추진력·목표지향. 기대/새로움 추구.\n"
    "- 세로토닌(serotonin): 안정/만족/평온·사회적 유대. 충동 억제·기분 균형.\n"
    "- 노르에피네프린(norepinephrine): 각성/집중/경계·에너지·스트레스 반응.\n"
    "- 멜라토닌(melatonin): 수면/암시성·생체리듬·야간 안정.\n\n"
    "지침:\n"
    "1) 입력의 정서, 의도, 맥락을 읽고 네 신경전달물질 경향을 정규화(0~1) 점수로 매긴다.\n"
    "2) 합은 1이 될 필요 없음. 네 축은 독립적이다.\n"
    "3) 출력은 정확히 두 줄만: 첫 줄은 NTL 포맷, 둘째 줄은 '이유: ' 설명. 다른 텍스트 금지.\n"
    "예시:\n"
    "입력: '밤새 코딩해서 데드라인 맞추고 싶어.'\n"
    "출력: [0.840, 0.210, 0.690, 0.050]([도파민, 세로토닌, 노르에피네프린, 멜라토닌])\n"
    "이유: 목표 달성 욕구와 각성/긴장이 높고, 안정/수면은 낮다.\n"
)

USER_WRAPPER = (
    "다음 문장을 NTL로 변환하라. 형식 엄수: 첫 줄은 [d, s, n, m]([도파민, 세로토닌, 노르에피네프린, 멜라토닌]), "
    "둘째 줄은 '이유: ' 한 문장.\n"
    "문장: "
)

# =============================
# 파싱/검증 유틸
# =============================
_float = r"(?:0(?:\.\d{1,3})?|1(?:\.0{1,3})?|0?\.\d{1,3}|0|1)"
STRICT_RE = re.compile(
    rf"\[\s*({_float})\s*,\s*({_float})\s*,\s*({_float})\s*,\s*({_float})\s*\]\(\[\s*도파민\s*,\s*세로토닌\s*,\s*노르에피네프린\s*,\s*멜라토닌\s*\]\)"
)

LABELS = ["도파민", "세로토닌", "노르에피네프린", "멜라토닌"]

def _clip01(x: float) -> float:
    return float(min(1.0, max(0.0, x)))

def parse_ntl(text: str):
    """
    모델 출력에서 NTL 벡터와 이유를 파싱하고 검증한다.
    반환: (vector: List[float], labels: List[str], reason: str)
    """
    raw = text.strip()
    lines = [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # 1) 라벨 포함 정규식으로 벡터 라인 우선 탐색
    vec = None
    for ln in lines:
        m = STRICT_RE.search(ln)
        if m:
            vec = [_clip01(float(g)) for g in m.groups()]
            break

    # 2) 형식이 어긋난 경우: 첫 []에서 숫자 4개만 추출
    if vec is None:
        for ln in lines:
            if "[" in ln and "]" in ln:
                m2 = re.search(r"\[([^\]]+)\]", ln)
                if not m2:
                    continue
                try:
                    nums = [float(x) for x in m2.group(1).split(",")]
                    nums = nums[:4]
                    if len(nums) == 4:
                        vec = [_clip01(float(x)) for x in nums]
                        break
                except Exception:
                    continue
        if vec is None:
            raise ValueError(f"출력 파싱 실패: {text}\n(raw=\n{raw}\n)")

    # 이유 라인 추출
    reason = ""
    for ln in lines:
        if ln.lower().startswith("이유:") or ln.lower().startswith("reason:"):
            reason = ln.split(":", 1)[1].strip()
            break

    return vec, LABELS, reason

# =============================
# 생성 파이프 설정
# =============================
@dataclass
class GenerationConfig:
    max_new_tokens: int = 48
    temperature: float = 0.0  # 결정적 생성
    top_p: float = 0.95
    repetition_penalty: float = 1.05
    stop: Tuple[str, ...] = ()

class LLM:
    def __init__(self, generator_pipeline, system_primer: str = SYSTEM_PRIMER):
        self.gen = generator_pipeline
        self.system = system_primer

    def _format_prompt(self, text: str) -> str:
        return self.system + "\n\n" + USER_WRAPPER + text.strip()

    def encode_ntl(self, text: str, config: GenerationConfig = GenerationConfig()) -> Dict[str, Any]:
        prompt = self._format_prompt(text)
        out_full = self.gen(
            prompt,
            max_new_tokens=config.max_new_tokens,
            temperature=config.temperature,
            do_sample=bool(config.temperature and config.temperature > 0.0),
            top_p=config.top_p,
            repetition_penalty=config.repetition_penalty,
            eos_token_id=_tok.eos_token_id,
            pad_token_id=_tok.pad_token_id,
        )[0]["generated_text"][len(prompt):]

        try:
            vec, labels, reason = parse_ntl(out_full)
        except Exception:
            # 최후 수단 복구: 숫자 4개만 긁어서 벡터 구성
            nums = re.findall(r"\d*\.\d+|\d+", out_full)
            if len(nums) >= 4:
                vec = [_clip01(float(x)) for x in nums[:4]]
                labels = LABELS
                reason = ""
            else:
                raise

        return {
            "input": text,
            "vector": vec,
            "labels": labels,
            "reason": reason,
            "formatted": f"[{vec[0]:.3f}, {vec[1]:.3f}, {vec[2]:.3f}, {vec[3]:.3f}]([도파민, 세로토닌, 노르에피네프린, 멜라토닌])",
            "raw": out_full.strip(),
        }

    def batch_encode_ntl(self, texts: List[str], config: GenerationConfig = GenerationConfig()) -> List[Dict[str, Any]]:
        return [self.encode_ntl(t, config) for t in texts]

# =============================
# 사용 예시 & 간단 테스트
# =============================
if __name__ == "__main__":
    llm = LLM(_gen)

    # 샘플 문장 4개
    samples = [
        "새로 시작한 프로젝트에 너무 설레고 집중이 잘 돼. 밤을 새워서라도 끝내고 싶어.",
        "요즘은 마음이 잔잔하고 친구들이랑 함께 있으면 안정돼.",
        "마감이 가까워져서 긴장되고 머리가 더 예민해진 느낌이야.",
        "불 꺼진 방에서 금방 잠들 준비가 됐어."
    ]

    # 1) 개별 실행 출력
    print("\n=== SAMPLE RUN ===")
    for s in samples:
        res = llm.encode_ntl(s)
        print(json.dumps({
            "input": res["input"],
            "formatted": res["formatted"],
            "reason": res.get("reason", "")
        }, ensure_ascii=False))

    # 2) 파서 회복력 테스트 (LLM 호출 없이 파서만)
    print("\n=== PARSER TESTS ===")
    noisy = """
    이런저런 서론... 무시해도 됨
    [0.8, 0.5, 0.12, 0.456]([도파민, 세로토닌, 노르에피네프린, 멜라토닌])
    이유: 테스트 문장 — 동기/보상 및 각성 경향.
    꼬릿말...
    """
    v, labels, r = parse_ntl(noisy)
    assert len(v) == 4 and all(0.0 <= x <= 1.0 for x in v)
    assert labels == ["도파민", "세로토닌", "노르에피네프린", "멜라토닌"]
    assert isinstance(r, str)

    unlabeled = """
    [0.3, 0.4, 0.2, 0.1]
    이유: 라벨이 없어도 복구.
    """
    v2, labels2, r2 = parse_ntl(unlabeled)
    assert len(v2) == 4 and all(0.0 <= x <= 1.0 for x in v2)
    assert labels2 == labels

    print("[TEST PASS] 기본 파서/출력 흐름 OK")
    


Device: cuda, dtype: torch.bfloat16


Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.34s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.08 GiB. GPU 0 has a total capacity of 23.99 GiB of which 0 bytes is free. Of the allocated memory 37.46 GiB is allocated by PyTorch, and 383.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)